<a href="https://colab.research.google.com/github/kimgu11/NLP_2023/blob/main/example/maze_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import random
import time

In [2]:
class Maze:  #미로 생성 코드
    view_delay_sec = 1.0

    def __init__(self, height, width, ratio=0.5):
        self.height = height
        self.width = width
        self.start_row, self.start_col = 1, 1
        self.end_row, self.end_col = height, width

        self._maze_init()
        self._fill(ratio)

    def _maze_init(self):
        hwall = ['#'] * (self.width + 2)
        self.maze = [hwall]
        for row in range(1, self.height + 1):
            self.maze.append(['#'] + [' '] * self.width + ['#'])
        self.maze.append(hwall)

    def _fill(self, ratio):
        empty_slots = []
        for row in range(1, self.height + 1):
            for col in range(1, self.width + 1):
                if (row, col) not in [(1, 1), (self.height, self.width)]:
                    empty_slots.append((row, col))

        n_blk = int(self.height * self.width * ratio)
        inner_walls = random.choices(empty_slots, k=n_blk)
        for row, col in inner_walls:
            self.maze[row][col] = '#'